使用神经网络解决Titanic问题，主要用两种实现，一种是纯pytorch实现神经网络的搭建，另一种是使用skorch包装好的以pytorch为后端的API实现。

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import time
import copy
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.nn.functional
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import warnings
import torch
torch.manual_seed(0)
warnings.filterwarnings("ignore")
print(torch.__version__)

0.4.1


读入数据文件，这里省去了特征工程的部分，特征采用之前特征工程提取出的特征。

In [2]:
train=pd.read_csv('train_features.csv')
test=pd.read_csv('test_features.csv')
train.head()

,PassengerId,Pclass,Sex,Title,Family_size,Family_Survival,FareBin_Code,AgeBin_Code,Embarked_C,Embarked_None,Embarked_Q,Embarked_S,Survived
0,1,3,0,0,2,0.5,0,2,0,0,0,1,0.0
1,2,1,1,2,2,0.5,4,3,1,0,0,0,1.0
2,3,3,1,1,1,0.5,1,2,0,0,0,1,1.0
3,4,1,1,2,2,0.0,4,3,0,0,0,1,1.0
4,5,3,0,0,1,0.5,1,3,0,0,0,1,0.0


In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 13 columns):
PassengerId        891 non-null int64
Pclass             891 non-null int64
Sex                891 non-null int64
Title              891 non-null int64
Family_size        891 non-null int64
Family_Survival    891 non-null float64
FareBin_Code       891 non-null int64
AgeBin_Code        891 non-null int64
Embarked_C         891 non-null int64
Embarked_None      891 non-null int64
Embarked_Q         891 non-null int64
Embarked_S         891 non-null int64
Survived           891 non-null float64
dtypes: float64(2), int64(11)
memory usage: 90.6 KB


In [4]:
train.describe()

,PassengerId,Pclass,Sex,Title,Family_size,Family_Survival,FareBin_Code,AgeBin_Code,Embarked_C,Embarked_None,Embarked_Q,Embarked_S,Survived
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,2.308642,0.352413,0.686869,1.904602,0.519641,1.985410,2.433221,0.188552,0.002245,0.086420,0.722783,0.383838
std,257.353842,0.836071,0.477990,0.972450,1.613459,0.323961,1.411355,1.370957,0.391372,0.047351,0.281141,0.447876,0.486592
min,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,223.500000,2.000000,0.000000,0.000000,1.000000,0.500000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,446.000000,3.000000,0.000000,0.000000,1.000000,0.500000,2.000000,2.000000,0.000000,0.000000,0.000000,1.000000,0.000000
75%,668.500000,3.000000,1.000000,1.000000,2.000000,0.500000,3.000000,3.000000,0.000000,0.000000,0.000000,1.000000,1.000000
max,891.000000,3.000000,1.000000,4.000000,11.000000,1.000000,4.000000,7.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [5]:
X_train=train.drop(['PassengerId','Survived'],axis=1).as_matrix()
Y_train=train['Survived'].astype(int).as_matrix()

X_test=test.drop(['PassengerId','Survived'],axis=1).as_matrix()
IDtest=test['PassengerId']

# scalar
scaler=MinMaxScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

# 使用Pytorch搭建模型

## 数据载入和处理

In [6]:
class TitanicFeaturesDataset(Dataset):
    def __init__(self,X,y=None,transform=None):
        self.X=X
        self.y=y
        self.transform=transform
    
    def __len__(self):
        return self.X.shape[0]
    
    def __getitem__(self,idx):
        if self.y is not None:
            sample={'X':self.X[idx],'y':self.y[idx]}
        else:
            sample={'X':self.X[idx]}
        if self.transform is not None:
            sample=self.transform(sample)
        return sample

In [7]:
# transform class
class ToTensor(object):
    def __call__(self,sample):
        if 'y' in sample.keys():
            X,y=sample['X'],sample['y']
            return {
                'X':torch.from_numpy(X.astype(np.float32)),
                'y':torch.squeeze(torch.from_numpy(np.array([y.astype(np.int64)])))
            }
        else:
            X=sample['X']
            return {
                'X':torch.from_numpy(X.astype(np.float32))
            }

In [8]:
# Model class
# 自定义Model需要继承nn.Module
class ClassifierModule(nn.Module):
    def __init__(self,D_in=11,D_out=2,num_units=20,nonlin=F.relu,dropout=0.5):
        super(ClassifierModule,self).__init__()
        self.num_units=num_units
        self.nonlin=nonlin
        self.dropout=nn.Dropout(dropout)
        
        self.linear1=nn.Linear(D_in,num_units)
        self.linear2=nn.Linear(num_units,10)
        self.output=nn.Linear(10,2)
        
    def forward(self,X):
        X=self.nonlin(self.linear1(X))
        X=self.dropout(X)
        X=self.nonlin(self.linear2(X))
        X=self.dropout(X)
        X=F.softmax(self.output(X),dim=-1)
        return X

In [9]:
# train model
def train_model(model,criterion,optimiizer,scheduler,dataset_sizes,num_epochs=100,device='cpu'):
    start=time.time()
    best_model_wts=copy.deepcopy(model.state_dict())
    best_acc=0.0
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch,num_epochs-1))
        print('-'*10)
        
        for phase in ['train','val']:
            if phase == 'train':
                scheduler.step()
                model.train()
            else:
                model.eval()
            running_loss=0.0
            running_corrects=0
            for sample_batches in dataloaders[phase]:
                inputs=sample_batches['X'].to(device)
                labels=sample_batches['y'].to(device)
                
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase=='train'):
                    outputs=model(inputs)
                    _,preds=torch.max(outputs,1)
                    loss=criterion(outputs,labels)
                    
                    if phase=='train':
                        loss.backward()
                        optimizer.step()
                    running_loss+=loss.item()*inputs.size(0)
                    running_corrects+=torch.sum(preds==labels.data)
            epoch_loss=running_loss/dataset_sizes[phase]
            epoch_acc=running_corrects.double()/dataset_sizes[phase]
            print('{} loss: {:.4f} Acc: {:.4f}'.format(phase,epoch_loss,epoch_acc))
            
            if phase=='val' and epoch_acc>best_acc:
                best_acc=epoch_acc
                best_model_wts=copy.deepcopy(model.state_dict())
    time_elapsed=time.time()-start
    print('Trainning complete in {:.0f}m {:.0f}s'.format(
            time_elapsed//60,time_elapsed%60))
    print('Best val Acc: {:4f}'.format(best_acc))
    model.load_state_dict(best_model_wts)
    return model              
                

In [10]:
# load data and train
train_dataset_len=X_train.shape[0]
train_len=train_dataset_len*4//5

transformed_datasets = {
        'train': TitanicFeaturesDataset(X_train[:train_len],Y_train[:train_len], transform=transforms.Compose([ToTensor()])),
        'val': TitanicFeaturesDataset(X_train[train_len:], Y_train[train_len:], transform=transforms.Compose([ToTensor()])),
        'test':TitanicFeaturesDataset(X_test,transform=transforms.Compose([ToTensor()]))
        }

dataloaders = {x: DataLoader(transformed_datasets[x], batch_size=16,
                                 shuffle=True, num_workers=0)
                   for x in ['train', 'val']}
dataloaders['test']=DataLoader(transformed_datasets['test'],batch_size=16,shuffle=False,num_workers=0)

dataset_sizes = {x: len(transformed_datasets[x]) for x in ['train', 'val','test']}
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


In [11]:
model=ClassifierModule()
criterion=nn.CrossEntropyLoss()
optimizer=optim.SGD(model.parameters(),lr=2e-2,momentum=0.9)
exp_lr_scheduler=lr_scheduler.StepLR(optimizer,step_size=7,gamma=0.1)

model=train_model(model,criterion,optimizer,exp_lr_scheduler,dataset_sizes,num_epochs=100,device=device)

Epoch 0/99
----------
train loss: 0.6819 Acc: 0.5674
val loss: 0.6576 Acc: 0.6425
Epoch 1/99
----------
train loss: 0.6624 Acc: 0.6096
val loss: 0.6418 Acc: 0.6425
Epoch 2/99
----------
train loss: 0.6570 Acc: 0.6096
val loss: 0.6249 Acc: 0.6425
Epoch 3/99
----------
train loss: 0.6463 Acc: 0.6096
val loss: 0.6069 Acc: 0.6425
Epoch 4/99
----------
train loss: 0.6344 Acc: 0.6110
val loss: 0.5795 Acc: 0.6425
Epoch 5/99
----------
train loss: 0.6212 Acc: 0.6236
val loss: 0.5534 Acc: 0.7933
Epoch 6/99
----------
train loss: 0.6010 Acc: 0.6966
val loss: 0.5272 Acc: 0.8380
Epoch 7/99
----------
train loss: 0.5842 Acc: 0.7177
val loss: 0.5245 Acc: 0.8380
Epoch 8/99
----------
train loss: 0.5920 Acc: 0.7233
val loss: 0.5218 Acc: 0.8380
Epoch 9/99
----------
train loss: 0.5844 Acc: 0.7219
val loss: 0.5188 Acc: 0.8380
Epoch 10/99
----------
train loss: 0.5748 Acc: 0.7654
val loss: 0.5160 Acc: 0.8436
Epoch 11/99
----------
train loss: 0.5846 Acc: 0.7275
val loss: 0.5138 Acc: 0.8436
Epoch 12/99
--

val loss: 0.5075 Acc: 0.8380
Epoch 99/99
----------
train loss: 0.5792 Acc: 0.7430
val loss: 0.5075 Acc: 0.8380
Trainning complete in 0m 7s
Best val Acc: 0.843575


In [12]:
# save model and reload it
torch.save(model.state_dict(),'pytorch_model.pth')
reloaded_model=ClassifierModule()
reloaded_model.load_state_dict(torch.load('pytorch_model.pth'))
# predicts on  test dataset

reloaded_model.eval()
final_predicts=[]
with torch.no_grad():
    for samples in dataloaders['test']:
        inputs=samples['X'].to(device)
        outputs=reloaded_model(inputs)
        _,preds=torch.max(outputs,1)
        for y_predict in list(preds.numpy()):
            final_predicts.append(y_predict)
final_predicts=np.array(final_predicts).reshape(-1,)
predict_survived_pytorch=pd.Series(final_predicts,name='Survived')
pytorch_result=pd.concat([IDtest,predict_survived_pytorch],axis=1)
pytorch_result.to_csv('pytorch_result.csv',index=False)
pytorch_result.head()
            

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1


# 使用skorch搭建模型

In [13]:
from skorch import NeuralNetClassifier
from skorch.dataset import CVSplit
from sklearn.model_selection import GridSearchCV
import skorch
print(skorch.__version__)
model = NeuralNetClassifier(
        ClassifierModule,
        max_epochs=200,
        lr=0.02,
        train_split=CVSplit(5),
        #     device='cuda',  # uncomment this to train with CUDA
    )
   

X = X_train.astype(np.float32)
Y = Y_train.astype(np.int64)

params = {
        'lr': [0.02],
        'max_epochs': [200],
        'module__num_units': [20],
}
gs = GridSearchCV(model, params, refit=True, cv=2, scoring='accuracy')
gs.fit(X, Y)
print(gs.best_score_, gs.best_params_)
gs.estimator.set_params(**gs.best_params_).fit(X, Y)

predict_Survived_skorch = pd.Series(gs.predict(X_test.astype(np.float32)), name='Survived')
skorch_result = pd.concat([IDtest, predict_Survived_skorch], axis=1)
skorch_result.to_csv('skorch_result.csv', index=False)
skorch_result.head()

0.3.0
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.7777       0.3596        0.7712  0.0215
      2        0.7674       0.3596        0.7610  0.0090
      3        0.7622       0.3596        0.7513  0.0145
      4        0.7513       0.3596        0.7424  0.0145
      5        0.7402       0.3596        0.7349  0.0125
      6        0.7366       0.3596        0.7277  0.0110
      7        0.7256       0.3596        0.7212  0.0110
      8        0.7245       0.3596        0.7153  0.0100
      9        0.7177       0.3596        0.7097  0.0125
     10        0.7100       0.3596        0.7046  0.0135
     11        0.7006       0.3820        0.7000  0.0105
     12        0.7112       0.3933        0.6957  0.0135
     13        0.6965       0.4157        0.6918  0.0120
     14        0.6926       0.4382        0.6881  0.0110
     15        0.6905       0.5618        0.6849  0.0125
     16        0.6847    

    142        0.5899       0.7753        0.5471  0.0080
    143        0.5901       0.7865        0.5462  0.0125
    144        0.5750       0.7865        0.5447  0.0140
    145        0.5878       0.7865        0.5435  0.0125
    146        0.5909       0.7865        0.5425  0.0090
    147        0.5714       0.7978        0.5410  0.0085
    148        0.5854       0.7978        0.5400  0.0105
    149        0.5767       0.7978        0.5390  0.0110
    150        0.5808       0.7978        0.5379  0.0110
    151        0.5680       0.7978        0.5365  0.0125
    152        0.5918       0.7978        0.5357  0.0130
    153        0.5818       0.8202        0.5346  0.0120
    154        0.5749       0.8202        0.5333  0.0135
    155        0.5736       0.8202        0.5323  0.0090
    156        0.5680       0.8202        0.5312  0.0085
    157        0.5671       0.8427        0.5298  0.0130
    158        0.5642       0.8427        0.5285  0.0195
    159        0.5744       0.8

     83        0.6376       0.5667        0.6552  0.0130
     84        0.6400       0.5667        0.6545  0.0100
     85        0.6378       0.5667        0.6539  0.0100
     86        0.6429       0.5667        0.6534  0.0155
     87        0.6344       0.5667        0.6528  0.0165
     88        0.6241       0.5667        0.6521  0.0125
     89        0.6505       0.5667        0.6517  0.0155
     90        0.6300       0.5667        0.6512  0.0120
     91        0.6369       0.5667        0.6509  0.0120
     92        0.6375       0.5667        0.6504  0.0125
     93        0.6346       0.5667        0.6499  0.0115
     94        0.6304       0.5667        0.6492  0.0125
     95        0.6393       0.5667        0.6488  0.0085
     96        0.6301       0.5667        0.6483  0.0115
     97        0.6365       0.5667        0.6477  0.0120
     98        0.6270       0.5667        0.6470  0.0090
     99        0.6280       0.5667        0.6463  0.0100
    100        0.6315       0.5

     24        0.6657       0.6704        0.6384  0.0195
     25        0.6674       0.6704        0.6382  0.0215
     26        0.6686       0.6704        0.6380  0.0190
     27        0.6668       0.6704        0.6378  0.0175
     28        0.6663       0.6704        0.6376  0.0205
     29        0.6696       0.6704        0.6374  0.0185
     30        0.6684       0.6704        0.6372  0.0190
     31        0.6633       0.6704        0.6370  0.0180
     32        0.6657       0.6704        0.6366  0.0200
     33        0.6664       0.6704        0.6363  0.0190
     34        0.6666       0.6704        0.6360  0.0165
     35        0.6640       0.6704        0.6358  0.0195
     36        0.6652       0.6704        0.6354  0.0165
     37        0.6649       0.6704        0.6352  0.0210
     38        0.6633       0.6704        0.6350  0.0190
     39        0.6649       0.6704        0.6347  0.0215
     40        0.6617       0.6704        0.6345  0.0195
     41        0.6622       0.6

    167        0.5753       0.7709        0.5211  0.0245
    168        0.5539       0.7709        0.5198  0.0250
    169        0.5768       0.7709        0.5192  0.0220
    170        0.5715       0.7709        0.5185  0.0205
    171        0.5520       0.7709        0.5176  0.0200
    172        0.5551       0.7709        0.5167  0.0220
    173        0.5649       0.7709        0.5157  0.0205
    174        0.5605       0.7709        0.5146  0.0195
    175        0.5507       0.7709        0.5136  0.0185
    176        0.5541       0.7654        0.5128  0.0200
    177        0.5628       0.7709        0.5121  0.0210
    178        0.5452       0.7654        0.5107  0.0264
    179        0.5745       0.7709        0.5103  0.0260
    180        0.5342       0.7654        0.5089  0.0245
    181        0.5584       0.7654        0.5083  0.0230
    182        0.5395       0.7654        0.5068  0.0225
    183        0.5463       0.7654        0.5055  0.0230
    184        0.5305       0.7

    107        0.6512       0.6704        0.6196  0.0210
    108        0.6600       0.6704        0.6194  0.0200
    109        0.6427       0.6704        0.6186  0.0200
    110        0.6519       0.6704        0.6181  0.0180
    111        0.6358       0.6704        0.6169  0.0180
    112        0.6501       0.6704        0.6164  0.0205
    113        0.6434       0.6704        0.6155  0.0195
    114        0.6409       0.6704        0.6144  0.0185
    115        0.6399       0.6704        0.6136  0.0210
    116        0.6430       0.6704        0.6128  0.0190
    117        0.6374       0.6704        0.6118  0.0180
    118        0.6368       0.6704        0.6108  0.0190
    119        0.6535       0.6704        0.6104  0.0205
    120        0.6338       0.6704        0.6093  0.0195
    121        0.6377       0.6704        0.6084  0.0225
    122        0.6298       0.6704        0.6071  0.0185
    123        0.6393       0.6704        0.6063  0.0200
    124        0.6252       0.6

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1


# 参考
1、[Pytorch data_loading tutorial](https://pytorch.org/tutorials/beginner/data_loading_tutorial.html#)

2、[Pytorch_transfer_learning_tutorial](https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html)

3、[skorch basic usage](https://nbviewer.jupyter.org/github/dnouri/skorch/blob/master/notebooks/Basic_Usage.ipynb)